# Results and Analysis

## Hierarchical Classification: Architecture Comparison

### Evaluation metrics: **Accuracy, Precision, Recall, F1-Score, AUC, ROC Curves**

In [ ]:
import sys
sys.path.insert(0, '..')

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from tqdm import tqdm

from config import (
    DEVICE, DATA_CONFIG, MODEL_CONFIG, PATHS, set_seed, DEFAULT_MERGED_DATASETS
)
from utils.data_loader import create_hierarchical_dataset, REGION_FINE_CLASS_COUNTS
from utils.hierarchical_model import HierarchicalClassificationModel

set_seed(42)
print(f"Device: {DEVICE}")

## 1. Load Test Data

In [ ]:
# Load test data
train_loader, val_loader, test_loader, dataset_info = create_hierarchical_dataset(
    datasets_to_include=DEFAULT_MERGED_DATASETS,
    batch_size=DATA_CONFIG['batch_size'],
    num_workers=DATA_CONFIG['num_workers']
)

print(f"Test samples: {dataset_info['test_samples']}")
print(f"Regions: {dataset_info['idx_to_region']}")

## 2. Discover Available Hierarchical Models

In [ ]:
ARCHITECTURES = ['enhanced', 'resnet18_3d', 'resnet34_3d', 'densenet121_3d', 'efficientnet3d_b0']

hierarchical_models = {}

print("Scanning for trained hierarchical models...")
print("-" * 50)

for arch in ARCHITECTURES:
    model_path = f"{PATHS['models']}/hierarchical_{arch}.pth"
    if os.path.exists(model_path):
        hierarchical_models[arch] = torch.load(model_path, map_location=DEVICE)
        print(f"Found: hierarchical_{arch}.pth")

print("-" * 50)
print(f"Models found: {list(hierarchical_models.keys())}")

In [ ]:
# Load hierarchical model
def compute_metrics(y_true, y_pred, y_proba=None, num_classes=None):
    """Compute accuracy, precision, recall, F1, and AUC."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1': f1_score(y_true, y_pred, average='weighted', zero_division=0),
    }
    
    # AUC (requires probability scores and multi-class handling)
    if y_proba is not None and num_classes is not None and num_classes > 1:
        try:
            if num_classes == 2:
                metrics['auc'] = roc_auc_score(y_true, y_proba[:, 1])
            else:
                y_true_bin = label_binarize(y_true, classes=range(num_classes))
                metrics['auc'] = roc_auc_score(y_true_bin, y_proba, average='weighted', multi_class='ovr')
        except Exception as e:
            metrics['auc'] = None
    else:
        metrics['auc'] = None
    
    return metrics

# Compute metrics for each architecture
all_metrics = {}

for arch, checkpoint in hierarchical_models.items():
    results = checkpoint['test_results']
    
    # Coarse (Stage 1) metrics
    coarse_metrics = compute_metrics(
        results['coarse_labels'],
        results['coarse_predictions'],
        num_classes=len(dataset_info['idx_to_region'])
    )
    
    # Fine (Stage 2) metrics  
    fine_metrics = compute_metrics(
        results['fine_labels'],
        results['fine_predictions']
    )
    
    all_metrics[arch] = {
        'coarse': coarse_metrics,
        'fine': fine_metrics,
        'per_region': results['fine_accuracy_per_region']
    }

In [ ]:
print("\n" + "="*100)
print("HIERARCHICAL MODEL COMPARISON - STAGE 1 (COARSE/REGION)")
print("="*100)

print(f"\n{'Architecture':<18} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'AUC':<12}")
print("-" * 100)

for arch in ARCHITECTURES:
    if arch not in all_metrics:
        continue
    m = all_metrics[arch]['coarse']
    auc_str = f"{m['auc']:.4f}" if m['auc'] else "N/A"
    print(f"{arch:<18} {m['accuracy']:<12.4f} {m['precision']:<12.4f} {m['recall']:<12.4f} {m['f1']:<12.4f} {auc_str:<12}")

print("\n" + "="*100)
print("HIERARCHICAL MODEL COMPARISON - STAGE 2 (FINE/PATHOLOGY)")
print("="*100)

print(f"\n{'Architecture':<18} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 100)

for arch in ARCHITECTURES:
    if arch not in all_metrics:
        continue
    m = all_metrics[arch]['fine']
    print(f"{arch:<18} {m['accuracy']:<12.4f} {m['precision']:<12.4f} {m['recall']:<12.4f} {m['f1']:<12.4f}")

print("-" * 100)

## 3. Results Comparison

In [ ]:
print("\n" + "="*100)
print("FINE ACCURACY PER ANATOMICAL REGION")
print("="*100)

print(f"\n{'Architecture':<18} {'Abdomen':<15} {'Chest':<15} {'Brain':<15}")
print("-" * 100)

for arch in ARCHITECTURES:
    if arch not in all_metrics:
        continue
    pr = all_metrics[arch]['per_region']
    print(f"{arch:<18} {pr.get('abdomen', 0):<15.4f} {pr.get('chest', 0):<15.4f} {pr.get('brain', 0):<15.4f}")

## 4. Training Curves Comparison

In [ ]:
if all_metrics:
    archs = list(all_metrics.keys())
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    x = np.arange(len(archs))
    width = 0.2
    
    # Stage 1 (Coarse) metrics
    for i, metric in enumerate(['accuracy', 'precision', 'recall', 'f1']):
        values = [all_metrics[a]['coarse'][metric] for a in archs]
        axes[0].bar(x + i*width, values, width, label=metrics_names[i])
    
    axes[0].set_xlabel('Architecture')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Stage 1 (Coarse) - Region Classification')
    axes[0].set_xticks(x + width*1.5)
    axes[0].set_xticklabels(archs, rotation=45, ha='right')
    axes[0].legend()
    axes[0].set_ylim(0, 1)
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Stage 2 (Fine) metrics
    for i, metric in enumerate(['accuracy', 'precision', 'recall', 'f1']):
        values = [all_metrics[a]['fine'][metric] for a in archs]
        axes[1].bar(x + i*width, values, width, label=metrics_names[i])
    
    axes[1].set_xlabel('Architecture')
    axes[1].set_ylabel('Score')
    axes[1].set_title('Stage 2 (Fine) - Pathology Classification')
    axes[1].set_xticks(x + width*1.5)
    axes[1].set_xticklabels(archs, rotation=45, ha='right')
    axes[1].legend()
    axes[1].set_ylim(0, 1)
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(f"{PATHS['figures']}/metrics_comparison.png", dpi=150)
    plt.show()

## 5. Confusion Matrices

In [ ]:
def plot_roc_curves(y_true, y_pred, class_names, title, save_path):
    """Plot ROC curves for multi-class classification."""
    n_classes = len(class_names)
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    y_pred_bin = label_binarize(y_pred, classes=range(n_classes))
    
    fig, ax = plt.subplots(figsize=(10, 8))
    colors = plt.cm.Set1(np.linspace(0, 1, n_classes))
    
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_bin[:, i])
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, color=color, lw=2, label=f'{class_name} (AUC = {roc_auc:.3f})')
    
    ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.500)')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(title)
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.show()

if all_metrics:
    # Find best model
    best_arch = max(all_metrics.keys(), key=lambda a: all_metrics[a]['fine']['f1'])
    results = hierarchical_models[best_arch]['test_results']
    
    print(f"\n ROC Curves for Best Model: {best_arch}")
    
    # Coarse ROC
    region_labels = [dataset_info['idx_to_region'][i] for i in range(len(dataset_info['idx_to_region']))]
    plot_roc_curves(
        results['coarse_labels'],
        results['coarse_predictions'],
        region_labels,
        f'ROC Curves - Stage 1 Coarse ({best_arch})',
        f"{PATHS['figures']}/roc_coarse_{best_arch}.png"
    )

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title, labels=None, save_path=None):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=labels, yticklabels=labels)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title(title)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150)
    plt.show()

if all_metrics:
    best_arch = max(all_metrics.keys(), key=lambda a: all_metrics[a]['fine']['f1'])
    results = hierarchical_models[best_arch]['test_results']
    
    # Coarse confusion matrix
    region_labels = [dataset_info['idx_to_region'][i] for i in range(len(dataset_info['idx_to_region']))]
    plot_confusion_matrix(
        results['coarse_labels'],
        results['coarse_predictions'],
        f'Confusion Matrix - Stage 1 ({best_arch})',
        labels=region_labels,
        save_path=f"{PATHS['figures']}/confusion_coarse_{best_arch}.png"
    )

## 6. Summary Table

In [ ]:
print("\n" + "="*100)
print("FINAL RANKING BY F1-SCORE")
print("="*100)

sorted_archs = sorted(all_metrics.keys(), key=lambda a: all_metrics[a]['fine']['f1'], reverse=True)

print(f"\n{'Rank':<6} {'Architecture':<18} {'Coarse F1':<12} {'Fine F1':<12} {'Fine Acc':<12}")
print("-" * 60)

for rank, arch in enumerate(sorted_archs, 1):
    m_c = all_metrics[arch]['coarse']
    m_f = all_metrics[arch]['fine']
    print(f"{rank:<6} {arch:<18} {m_c['f1']:<12.4f} {m_f['f1']:<12.4f} {m_f['accuracy']:<12.4f}")

print("-" * 60)

if sorted_archs:
    best = sorted_archs[0]
    m = all_metrics[best]
    print(f"\n BEST MODEL: {best}")
    print(f"   Stage 1 - Accuracy: {m['coarse']['accuracy']:.4f}, F1: {m['coarse']['f1']:.4f}")
    print(f"   Stage 2 - Accuracy: {m['fine']['accuracy']:.4f}, F1: {m['fine']['f1']:.4f}")